In [ ]:
import os
import openai
import numpy as np
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone, Index

print("Chatbot Notebook: Retrieval + GPT")

In [ ]:
# 1) Use environment variables for keys
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")

if not OPENAI_API_KEY:
    raise ValueError("Missing OPENAI_API_KEY environment variable.")
if not PINECONE_API_KEY:
    raise ValueError("Missing PINECONE_API_KEY environment variable.")

# 2) set openai key
openai.api_key = OPENAI_API_KEY

# 3) connect to pinecone data-plane
INDEX_NAME = "chatbot-index"
pc = Pinecone(api_key=PINECONE_API_KEY)
desc = pc.describe_index(INDEX_NAME)
host = desc.host
index = Index(api_key=PINECONE_API_KEY, host=host)

# 4) sentence transformers model
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
# 5) simple chatbot function
def chatbot_loop():
    print("Chatbot ready! Type 'exit' or 'quit' to stop.\n")
    while True:
        user_input = input("User: ")
        if user_input.lower() in ["exit", "quit"]:
            print("Goodbye!")
            break

        # embed
        q_vec = model.encode(user_input).tolist()
        results = index.query(
            vector=q_vec,
            top_k=3,
            include_metadata=True
        )

        # gather chunks
        retrieved_chunks = []
        for match in results.get("matches", []):
            meta = match.get("metadata")
            if meta:
                retrieved_chunks.append(meta.get("text", ""))

        context_text = "\n\n".join(retrieved_chunks)

        prompt = f"""
        You are an expert assistant. Use ONLY the following context to answer the user's question.
        Context:
        {context_text}

        Question:
        {user_input}

        If the answer is not found in the context, reply with: "I don't know based on the research data provided." 
        """

        try:
            resp = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "system", "content": prompt}],
                temperature=0.7
            )
            answer = resp.choices[0].message.content
        except Exception as e:
            answer = f"Error calling OpenAI API: {e}"

        print(f"Chatbot: {answer}\n{'-'*60}\n")

In [ ]:
# 6) run it!
chatbot_loop()